# Community detection using Spectral Clustering

Given graph $G$ with $n$ nodes, find non-overlapping node "communities": $k$ groups of nodes that are densely intra connected and have low number of inter connections.

- Compute square diagonal matrix of node degrees $D$. 
    $$D_{ii} = \sum_i A_{ij}, D_{ij} = 0, i \neq j$$
- Construct graph Laplacian 
    $$L_{unnormed} = D - A$$
Find $0 = \lambda_0 \leq \lambda_1 \leq \lambda_2 \leq \ldots \leq \lambda_m$ smallest eigenvalues of $L$ and construct matrix $X$ by stacking $m$ corresponding eigenvectors ($v_1, \ldots v_m$) as columns of $X$. Matrix $X$ has size $n \times m$, its rows are "spectral representaion" of graph nodes.
   
- Run k-means algorithm on matrix X and assign nodes with labels obtained by k-means.

---

Use 3 versions of a Laplacian:

- Unnormalized Laplacian: $L = D - A$
- Symmetric normalization: $L_{sym} = I - D^{-\frac{1}{2}} \cdot A \cdot D^{-\frac{1}{2}}$
- Random Walk normalization: $L_{rw} = I - D^{-1} \cdot A$
    
Sources:
1. Andrew Ng paper on spectral clustering https://ai.stanford.edu/~ang/papers/nips01-spectral.pdf
2. Tutorial on spectral clustering with multiple theoretical views on the problem https://arxiv.org/abs/0711.0189
3. Amazing explanation from James R. Lee https://www.youtube.com/watch?v=8XJes6XFjxM

## 1. Compare 3 versions of Spectral clustering on `Karate Club dataset`
    
Implement 3 algorithms described in https://arxiv.org/abs/0711.0189 :
1. Unnormalized spectral clustering 
2. Normalized spectral clustering according to Shi and Malik (2000)
3. Normalized spectral clustering according to Ng, Jordan, and Weiss (2002)

using `Adjusted Rand Index` (3 pairwise comparisons), `Modularity` (3 numbers) and visually, plotting points in a corresponding 2 dimensional spaces (spanned by eigenvectors).

Theoretical questions:
4. Why does the smallest eigenvalue of unnormalized Laplacian is always equal to 0? 
5. From network point of view, what does Symmetric normalization do? 
6. Under what conditions Symmetric and Random walk normalizations yield the same result? 
  

# Unsupervised node prediction

## 2. Random Walk based embeddings

Use data from https://github.com/shchur/gnn-benchmark/tree/master/data/npz , particulary use Amazon Computers dataset, see description in a corresponding paper https://arxiv.org/pdf/1811.05868.pdf. 
> Recall that this network has small amount of isolated nodes.

1. Run `DeepWalk` to get embeddings of size 32.
2. Using `kmeans` with appropriate number of clusters (somewhere between 6 and 12) compute node labels. Compare the result with ground truth communities using `adjusted_rand_index`. Compute corresponding `Modularities`.
3. Run your favourite dimensionality reduction algorithm to get a 2 dimensional embedding.
4. Compare results (repeate 1-3) with embeddings of size 64 and 128.
5. Compare results with supervised Logistic Regression on data feature matrix (without using network data).


Recommended for visualization:

- Dmitry Ulyanov has nice multicore tsne implementation https://github.com/DmitryUlyanov/Multicore-TSNE
- Recent paper from Aleksandr Artemenkov and Maxim Panov https://arxiv.org/pdf/2001.11411.pdf with implementation https://github.com/alartum/ncvis . Reported to be superior to TSNE for the purpose of 2 dimensional visualization.

In [1]:
import numpy as np

In [2]:
def load_npz_to_sparse_graph(file_name):
    """Load a SparseGraph from a Numpy binary file.
    from https://github.com/shchur/gnn-benchmark/blob/master/gnnbench/data/io.py
    """
    from scipy.sparse import csr_matrix
    with np.load(file_name) as loader:
        loader = dict(loader)
        adj_matrix = csr_matrix((loader['adj_data'], loader['adj_indices'], loader['adj_indptr']),
                                   shape=loader['adj_shape'])
        attr_matrix = csr_matrix((loader['attr_data'], loader['attr_indices'], loader['attr_indptr']),
                                        shape=loader['attr_shape'])
        labels = loader['labels']
        class_names = loader.get('class_names')

    return adj_matrix, attr_matrix, labels, class_names

In [3]:
adj, X, y, class_names = load_npz_to_sparse_graph('amazon_electronics_computers.npz')

In [4]:
adj.todense().shape

(13752, 13752)

# 3. Отправка решения.


1. Прогоните свой jupyter notebook: **Kernel** $\rightarrow$ **Restart & Run all**, нотбук **должен** запускаться линейно! Если вы считали эмбединги прямо в нотбуке, можете закоментить соответствующие ячейки. Во втором домашнем задании я **буду** снижать оценку если вы не сделали Restart & Run All.
2. Назовите нотбук `Имя_Фамилия_HA2_networks_2020`, например `Anvar_Kurmukov_HA2_networks_2020`
4. Сохраните нотбук в формате `ipython`.
5. Отправьте файл с нотбуком на почту kurmukovai@gmail.com с темой письма `Имя_Фамилия_HA2_networks_2020`.

---

## Комментарии к решению и выставлению оценки.
Ваш файл с решением должно быть **комфортно читать**:
- **комментируйте** происходящее, лучше всего для этого подходит markdown cells, но можно и в комментариях в ячейках с кодом. Под комментируйте имеется ввиду написание выводов на естественном языке доступном проверяющему (русский или английский). Например, вы нарисовали двумерное вложение и на картинке ничего не понятно, все смешано, это нормально (хотя во втором пункте можно получить более менее неплохое вложение), так и напишите "Не получилось построить...".
- не нужно оставлять все отладочные ячейки нотбука, сохраните свою работу отдельно, а в файле которые отправляете в качестве решения оставьте только нужное.
- **округляйте** значения характеристик точности и модулярности до нескольких значащих цифр (столько сколько целесообразно в каждом отдельном случае), но не более 6.